In [1]:
from keras import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd, numpy as np, matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Flatten, Dense

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.layers import Embedding

Using TensorFlow backend.


In [2]:
data = pd.read_csv(r"C:\Users\faeze\Desktop\train.csv", encoding='iso-8859-1')

In [3]:
X = data.SentimentText
y = data.Sentiment

In [8]:
labels=[]
for i in y:
    labels.append(i)

In [4]:
texts = []
for j in X:
    texts.append(j)

In [5]:
max_words = 10000
maxlen = 100

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
word_index = tokenizer.word_index

In [6]:
data = pad_sequences(sequences , maxlen=maxlen)

In [9]:
labels = np.asarray(labels)

In [12]:
indices = np.arange(data.shape[0])

np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2, random_state=100)

# اینجا می تونی هم مثل بالا داده رو به تنسور تبدیل کنی هم مثل کد پایین

In [ ]:
X = preprocessing.sequence.pad_sequences(sequences, maxlen=maxlen)

In [ ]:
np.random.shuffle(X)
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2, random_state=100)

In [ ]:
x_val = X_train[:1000]
partial_x_train = X_train[1000:]
y_val = y_train[:1000]
partial_y_train = y_train[1000:]

In [ ]:
labels = np.asarray(labels)

# ... 

In [ ]:
def create_embedding_matrix(filepath, word_index, embedding_dim):
    vocab_size = len(word_index) + 1
    embedding_matrix = np.zeros((vocab_size, embedding_dim))

    with open(filepath) as f:
        for line in f:
            word, *vector = line.split()
            if word in word_index:
                idx = word_index[word] 
                embedding_matrix[idx] = np.array(
                    vector, dtype=np.float32)[:embedding_dim]

    return embedding_matrix

In [ ]:
embedding_dim = 50
embedding_matrix = create_embedding_matrix(
    'data/glove_word_embeddings/glove.6B.100d.txt',
    tokenizer.word_index, embedding_dim)

In [ ]:
model = Sequential()
model.add(layers.Embedding(vocab_size, embedding_dim, weights=[embedding_matrix], input_length=maxlen, trainable=False))
model.add(layers.GlobalMaxPool1D())
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

# بالایی ترین ایبل فالس هست. در ادامه ی مدل دیگه هم گذاشتم که ترین ایبل ترو هست.

In [ ]:
history = model.fit(partial_x_train, partial_y_train,
                    epochs=5,
                    validation_data=(x_val, y_val),
                    batch_size=512)

loss, accuracy = model.evaluate(X_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

#  یا کد زیر: و به همراه اون کدایی که برای نمودار هست
#result = model.evaluate(X_test, y_test)
#result

In [ ]:
model = Sequential()
model.add(layers.Embedding(vocab_size, embedding_dim, weights=[embedding_matrix], input_length=maxlen, trainable=True))
model.add(layers.GlobalMaxPool1D())
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

In [ ]:
history = model.fit(partial_x_train, partial_y_train,
                    epochs=5,
                    validation_data=(x_val, y_val),
                    batch_size=512)
loss, accuracy = model.evaluate(X_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

# اینجا هم ، می تونی کدای بالا رو اجرا کنی و یا اینکه کدای زیر رو که کدای استاد هست.

In [ ]:
glove_dir = ''
embeddings_index = {}
f = open(os.path.join(glove_dir), 'glove.6B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:] , dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [ ]:
embedding_dim =100
embedding_matrix = np.zeros((max_words , embedding_dim))
for word , i in word_index.items():
    if i<max_words:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

In [ ]:
model = Sequential()
model.add(Embedding(max_words , embedding_dim , input_length=maxlen))
model.add(Flatten())
# گذاشتن لایه دراپ اوت
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.summary()

In [ ]:
model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable = False

In [ ]:
model.compile(optimizer = 'rmsprop',loss='binary_crossentropy', metrics=['acc'])

history = model.fit(partial_x_train, partial_y_train, epochs=5, batch_size=512, validation_data=(x_val, y_val))

In [ ]:
result = model.evaluate(X_test, y_test)
result

In [ ]:
history_dict = history.history
loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']

epochs = range(1, len(loss_values)+1)
plt.plot(epochs, loss_values, 'bo', label='Training Loss')
plt.plot(epochs, val_loss_values, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()
plt.show()

In [ ]:
history_dict = history.history
loss_values = history_dict['acc']
val_loss_values = history_dict['val_acc']

epochs = range(1, len(loss_values)+1)
plt.plot(epochs, loss_values, 'bo', label='Training Loss')
plt.plot(epochs, val_loss_values, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend()
plt.show()